## Декораторы функций

Создадим функцию func_decorator с параметром func, в который будет попадать ссылка на функцию. Внутри этой функции создадим внутреннюю функцию с именем wrapper. Эта функция будет вызывать ту функцию, ссылка на которую была передана на внешнюю функцию, но до и после может выполнять какие либо действия. В самом конце внешняя функция будет возвращать ссылку на внутреннюю. Получился декоратор в самом простом его виде:

In [1]:
def func_decorator(func):
    def wrapper():
        print("Действие перед запуском функции")
        func() # запуск функции
        print("Действие после запуска функции")
    return wrapper


Создадим вторую функцию, которая просто будет выводить в консоль строку "Тестовая функция":

In [2]:
def test_func():
    print("Тестовая функция")

Эту функцию можно вызвать самым простым образом:

In [3]:
test_func()

Тестовая функция


А можно и с помощью декоратора. Вызовем функцию func_decorator и передадим ей ссылку на функцию test_func. Сохраним результат (ссылку на внутреннюю функцию) в переменную f:

In [4]:
f = func_decorator(test_func)

И вызовем эту функцию:

In [5]:
f()

Действие перед запуском функции
Тестовая функция
Действие после запуска функции


Теперь помимо самой функции выполнились некие действия до и после.

Но обычно при декорировании создают ссылку на то же самое имя:

In [6]:
test_func = func_decorator(test_func)

In [7]:
test_func()

Действие перед запуском функции
Тестовая функция
Действие после запуска функции


В итоге функция test_func меняет свою работу. Она не просто выводит что либо в консоль: будут дополнительно выполнены действия до и после. И все это произошло благодаря механизму замыкания. 

Если мы хотим, чтобы функция test_func выводила в консоль пережанные ей значения, придется модифицировать и функцию, и декоратор:

In [8]:
def func_decorator(func):
    def wrapper(val):
        print("Действие перед запуском функции")
        func(val) # запуск функции
        print("Действие после запуска функции")
    return wrapper

def test_func(val):
    print(f"Тестовая функция выводит переданное ей значение - {val}")

In [9]:
test_func = func_decorator(test_func)

In [10]:
test_func("Привет!")

Действие перед запуском функции
Тестовая функция выводит переданное ей значение - Привет!
Действие после запуска функции


Но в этом случае декоратор не универсален: если изменится функция test_func, то придется редактировать и декоратор. Чтобы исправить это, в декораторе стоит использовать коллекции неименованных и неименованных параметров args и kwargs:

In [11]:
def func_decorator(func):
    def wrapper(*args, **kwargs):
        print("Действие перед запуском функции")
        func(*args, **kwargs) # запуск функции
        print("Действие после запуска функции")
    return wrapper

Которые будут запаковывать и распаковывать переданные функции test_func значения:

In [12]:
def test_func(val):
    print(f"Тестовая функция выводит переданное ей значение - {val}")

In [13]:
test_func = func_decorator(test_func)

In [14]:
test_func("Привет!")

Действие перед запуском функции
Тестовая функция выводит переданное ей значение - Привет!
Действие после запуска функции


И если функция изменится:

In [15]:
def test_func(val, val2):
    print(f"Тестовая функция выводит первое переданное ей значение - {val} и второе - {val2}")

Декоратор уже изменять не придется:

In [16]:
test_func = func_decorator(test_func)

In [17]:
test_func("Первый привет!", "Второй привет!")

Действие перед запуском функции
Тестовая функция выводит первое переданное ей значение - Первый привет! и второе - Второй привет!
Действие после запуска функции


Пример реального применения декоратора. Допустим, необходимо протестировать скорость работы различных функций. Допустим, это будет медленный и быстрый алгоритм Евклида для нахождения НОД:

In [18]:
def get_nod(a, b):
    while a != b:
        if a > b:
            a -= b
        else:
            b -= a
    return a

In [19]:
def get_nod_f(a, b):
    if a < b:
        a, b = b, a
    
    while b != 0:
        a, b = b, a % b
    return a

In [20]:
import time

Определим декоратор:

In [21]:
def time_deco(nod_f):
    def wrapper(*args):
        s_time = time.time()
        res = nod_f(*args)
        e_time = time.time()
        print(f"Время работы функции - {e_time - s_time}")
        return res
    return wrapper

In [22]:
get_nod = time_deco(get_nod)
get_nod_f = time_deco(get_nod_f)

В итоге помимо результата мы получаем и скорость работы функции:

In [23]:
get_nod(1000000, 2)

Время работы функции - 0.03279566764831543


2

In [24]:
get_nod_f(1000000, 2)

Время работы функции - 0.0


2

Причем этот декоратор универсален - с помощью него можно измерять скорость работы любых функций.

На практике декораторы применяются другим образом. Перед функцией ставится символ "@" и просписывается имя декоратора. Например, измерим скорость работы другой функции:

In [25]:
def calcFactorial(num):
    if num < 0: return None
    elif num == 0: return 1
    else: return num * calcFactorial(num - 1)

In [26]:
calcFactorial(4)

24

Прописав декоратор перед именем функции:

In [27]:
@time_deco
def calcFactorial(num):
    if num < 0: return None
    res = 1
    for x in range(1, num+1):
        res *= x
    return res

In [28]:
fact = calcFactorial(100000)

Время работы функции - 2.522308588027954
